In [1]:
from pymongo import MongoClient

In [2]:
# Create connection with Mongo to Get the Data
client = MongoClient('mongodb://13.126.59.19:27018/products')
db = client.products
products = db.products

In [3]:
# Get the data into a list
data = list()
for record in products.find():
     data.append(record)

In [4]:
# Let's see the first object.
data[0]

{'_id': ObjectId('5c41e82d0385c8710e4ae547'),
 'id': 'e7a780c3-502c-405a-9d2c-9e9eb0b638f5',
 'sku': 'ae3412ff-3693-45fe-ac24-2a23fad40a9e',
 'name': 'Mum Disbud XmasGreen Regular Wide 110 Cm Procona ',
 'shortdescription': '',
 'description': '',
 'group': 'Fresh flowers',
 'subgroup': 'Mum Disbud',
 'variety': 'H30',
 'grade': 'Regular Wide',
 'color': 'XmasGreen',
 'length': '110 Cm',
 'boxtype': 'Procona ',
 'active': False,
 'images': ['https://s3.ap-south-1.amazonaws.com/aob-deverp/rose-images/Freedom.jpg',
  'https://s3.ap-south-1.amazonaws.com/aob-deverp/rose-images/Marlyse+Rose.jpg',
  'https://s3.ap-south-1.amazonaws.com/aob-deverp/rose-images/Marlyse+Rose.jpg'],
 'xxx_nounkeyedliteral': {},
 'xxx_unrecognized': b'',
 'xxx_sizecache': 0}

In [32]:
import pandas as pd
import numpy as np

In [6]:
import re

In [7]:
def get_number(x, i=-1):
    numbers = re.findall(r'\d+', x)
    if len(numbers)>0:
        return numbers[i]
    return 0

In [8]:
def get_length(x):
    length = 0
    inch = re.search(r'\binch\b|\binches\b|\bin\b',x.lower())
    if inch:
        length = get_number(x[:inch.start()])
        return float(length)*2.54
    ft = re.search(r'\bfeet\b|\bfeets\b|\bft\b',x.lower())
    if ft:
        length = get_number(x[:ft.start()])
        return float(length)*30.48
    cm = re.search(r'\bcm\b|\bcentimeters\b|\bcentimeter\b|\bcenti meter\b|\bcenti meters\b',x.lower())
    if cm:
        length = get_number(x[:cm.start()])
        return length
    return get_number(x, 0)

In [9]:
features_to_use = ['id', 'shortdescription', 'description', 'group', 'subgroup', 'variety', 'grade', 'color', 'length', 'boxtype']

In [10]:
def tokenize_on_caps(x):
    return ' '.join(re.findall('[A-Z,(1-9)][^A-Z]*', x))

In [11]:
df = pd.DataFrame(data, columns=features_to_use)

In [12]:
df['length'] = df['length'].apply(lambda x: get_length(x))
df['color'] = df['color'].apply(lambda x: tokenize_on_caps(x))

In [13]:
df['all'] = df[features_to_use[1:]].apply(lambda x: ' '.join(x.map(str)), axis=1)

In [14]:
df['all'] = df['all'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x.strip()))

In [15]:
df.head(2)

,id,shortdescription,description,group,subgroup,variety,grade,color,length,boxtype,all
0,e7a780c3-502c-405a-9d2c-9e9eb0b638f5,,,Fresh flowers,Mum Disbud,H30,Regular Wide,Xmas Green,110,Procona,Fresh flowers Mum Disbud H30 Regular Wide Xmas...
1,ea3ed24e-8912-4424-a517-88202697b094,,,Collateral materials,Curly,Minuet,Medium Narrow,Off White,65,,Collateral materials Curly Minuet Medium Narro...


In [16]:
#Tokenizer
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load('en')
tokenizer = Tokenizer(nlp.vocab)

In [17]:
doc1 = nlp(df['all'][0])

In [18]:
doc1

Fresh flowers Mum Disbud H30 Regular Wide Xmas Green 110 Procona

In [19]:
df['all'][462]

'Fresh flowers Daisy Luna Regular Wide Lavender Dyed Yellow 120 Hamper'

In [20]:
highest_similarity = list()

In [21]:
for i in range (len(df)):
    docN = nlp(df['all'][i])
    similarity = doc1.similarity(docN) 
    #print(i, similarity)
    if similarity > 0.94:
        highest_similarity.append((i, similarity))

In [22]:
highest_similarity

[(0, 1.0), (462, 0.9409930804394734)]

In [23]:
df_for_processing = df[['id', 'all']]

In [28]:
df_for_processing['all'].head(3)

0    Fresh flowers Mum Disbud H30 Regular Wide Xmas...
1    Collateral materials Curly Minuet Medium Narro...
2    Premade bouquets Wreath Magical Bluebells Doub...
Name: all, dtype: object

In [93]:
for token in doc1:
    print(str(token.text) + " " + str(token.lemma_))

Fresh fresh
flowers flower
Mum mum
Disbud disbud
H30 h30
Regular regular
Wide wide
Xmas xmas
Green green
110 110
Procona procona


In [95]:
df.to_csv('./processed_product_data.csv')